In [4]:
import numpy as np
import networkx as nx
import DNGR_utils as ut
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, noise
from keras.models import Model

ImportError: cannot import name 'noise' from 'keras.layers' (D:\software\Lib\site-packages\keras\layers\__init__.py)

In [3]:
pip install tensorflow

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    unknown package:
        Expected sha256 4710b0ea84defaafc0d6cc51162ebef8b07da015fc68375661451861c5bf4421
             Got        d7defa1e1e55f522cd9be26a9871b6ab53278e516a53b66cff82cacc887ebe79



  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/fa

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata (5.1 kB)
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.26

In [2]:
!pip install matplotlib

  Using cached matplotlib-3.7.1-cp310-cp310-win_amd64.whl (7.6 MB)
  Using cached contourpy-1.0.7-cp310-cp310-win_amd64.whl (162 kB)
     ---------------------------------------- 0.0/98.3 kB ? eta -:--:--
     ------- ------------------------------ 20.5/98.3 kB 330.3 kB/s eta 0:00:01
     ----------- -------------------------- 30.7/98.3 kB 445.2 kB/s eta 0:00:01
     --------------- ---------------------- 41.0/98.3 kB 245.8 kB/s eta 0:00:01
     ----------------------- -------------- 61.4/98.3 kB 328.2 kB/s eta 0:00:01
     -------------------------------------- 98.3/98.3 kB 434.0 kB/s eta 0:00:00
  Using cached fonttools-4.39.3-py3-none-any.whl (1.0 MB)
     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     - -------------------------------------- 0.1/2.5 MB 1.6 MB/s eta 0:00:02
     - -------------------------------------- 0.1/2.5 MB 1.3 MB/s eta 0:00:02
     - -------------------------------------- 0.1/2.5 MB 1.3 MB/s eta 0:00:02
     --- -----------------------

In [4]:
#Stage 1 -  Random Surfing
def random_surf(cosine_sim_matrix, num_hops, alpha):
    
    num_nodes = len(cosine_sim_matrix)
    
    adj_matrix = ut.scale_sim_matrix(cosine_sim_matrix)
    P0 = np.eye(num_nodes, dtype='float32')
    P = np.eye(num_nodes, dtype='float32')
    A = np.zeros((num_nodes,num_nodes),dtype='float32')
    
    for i in range(num_hops):
        P = (alpha*np.dot(P,adj_matrix)) + ((1-alpha)*P0)
        A = A + P

    return A

In [5]:
#Stage 2 - PPMI Matrix
def PPMI_matrix(A):
    
    num_nodes = len(A)
    A = ut.scale_sim_matrix(A)
    
    row_sum = np.sum(A, axis=1).reshape(num_nodes,1)
    col_sum = np.sum(A, axis=0).reshape(1,num_nodes)
    
    D = np.sum(col_sum)
    PPMI = np.log(np.divide(np.multiply(D,A),np.dot(row_sum,col_sum)))
    #Gotta use numpy for division, else it runs into divide by zero error, now it'll store inf or -inf
    #All Diag elements will have either inf or -inf.
    #Get PPMI by making negative values to 0
    PPMI[np.isinf(PPMI)] = 0.0
    PPMI[np.isneginf(PPMI)] = 0.0
    PPMI[PPMI<0.0] = 0.0
    
    return PPMI

In [6]:
def sdae(PPMI, hidden_neurons):
    
    inp = Input(shape=(PPMI.shape[1],))
    enc = noise.GaussianNoise(0.2)(inp)
    
    for neurons in hidden_neurons:
        enc = Dense(neurons, activation = 'relu')(enc)
    
    dec = Dense(hidden_neurons[-2], activation = 'relu')(enc)
    for neurons in hidden_neurons[:-3][::-1]:
        dec = Dense(neurons, activation = 'relu')(dec)
    dec = Dense(PPMI.shape[1], activation='relu')(dec)
    
    auto_enc = Model(inputs=inp, outputs=dec)
    auto_enc.compile(optimizer='adam', loss='mse')
    
    auto_enc.fit(x=PPMI, y=PPMI, batch_size=10, epochs=5)
    
    encoder = Model(inputs=inp, outputs=enc)
    encoder.compile(optimizer='adam', loss='mse')
    embeddings = encoder.predict(PPMI)
    
    return embeddings

In [9]:
"""
- group can be  'NG3', 'NG6', 'NG9'
- Max hops for transition matrix - should be a positive natural number
- alpha is probability that random surfing will go to next vertex. (1-Alpha) is probability that it will return back to 
    original vertex and restart. Range 0-1
- hidden_neurons is the number of hidden neurons in each layer. Make sure there's 3 or more layers.
"""

group = 'NG6'
max_hops = 2
alpha = 0.98
hidden_neurons = [512,256,128]

text_corpus, file_names, target = ut.read_data(group)
cosine_sim_matrix = ut.get_cosine_sim_matrix(text_corpus)

A = random_surf(cosine_sim_matrix, max_hops, alpha)
PPMI = PPMI_matrix(A)

embeddings = sdae(PPMI, hidden_neurons)

#Save embeddings

#Metrics
ut.compute_metrics(embeddings, target)

#visualize embeddings using tsne
ut.visualize_TSNE(embeddings, target)
plt.show()


HTTPError: HTTP Error 403: Forbidden